In [12]:
import os
import av
import numpy as np
import cv2
import csv

r = 4  # Number of grid rows/columns
n = 12  # Number of bins for histograms

# Folder path containing videos
folder_path = 'Target_video/Wave'
process_videos_in_folder(folder_path)

# Process videos in the given folder
def process_videos_in_folder(folder_path):
    parent_folder = os.path.abspath(os.path.join(folder_path, os.pardir))
    
    output_base_folder = os.path.join(parent_folder, "Wave_output")
    os.makedirs(output_base_folder, exist_ok=True)

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".mp4") or file.endswith(".avi"):
                video_file = os.path.join(root, file)
                video_name = os.path.splitext(os.path.relpath(video_file, folder_path))[0]
                process_video(video_file, video_name, output_base_folder)

# Function to extract frames from the video
def frames_extracted(video_file):
    try:
        container = av.open(video_file)
        frames = [frame for frame in container.decode(video=0)]
        first_frame = frames[0]
        middle_frame = frames[len(frames) // 2]
        last_frame = frames[-1]
        return [first_frame, middle_frame, last_frame]
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError: Skipping file {video_file}. Error: {e}")
        return None
    except Exception as e:
        print(f"Error processing file {video_file}. Error: {e}")
        return None

# Process the video frames
def process_video(video_file, video_name, output_base_folder):
    frames = frames_extracted(video_file)
    if frames is None:
        print(f"Skipping {video_file} due to extraction failure.")
        return
    output_folder = os.path.join(output_base_folder, f'{video_name}_output')
    os.makedirs(output_folder, exist_ok=True)
    for frame_id, frame in enumerate(frames, 1):
        color_histograms_final(frame, r, n, frame_id, output_folder)
        
# Function to calculate and save color histograms in a CSV file
def color_histograms_final(frame, r, n, frame_id, output_folder):
    try:
        img = np.array(frame.to_image())
    except Exception as e:
        return None
    
    h, w, _ = img.shape
    cell_h, cell_w = h // r, w // r
    histogram_vectors = []

    for i in range(r):
        for j in range(r):
            cell = img[i * cell_h:(i + 1) * cell_h, j * cell_w:(j + 1) * cell_w]
            hist_r = cv2.calcHist([cell], [0], None, [n], [0, 256]).flatten()
            hist_g = cv2.calcHist([cell], [1], None, [n], [0, 256]).flatten()
            hist_b = cv2.calcHist([cell], [2], None, [n], [0, 256]).flatten()

            # Normalize histograms
            hist_r, hist_g, hist_b = hist_r / hist_r.sum(), hist_g / hist_g.sum(), hist_b / hist_b.sum()

            # Combine the RGB histograms
            histogram_final = [sum(x) for x in zip(hist_r, hist_g, hist_b)]
            histogram_vectors.append(histogram_final)
    # Write the histogram vectors to a CSV file
    csv_file = os.path.join(output_folder, f'{output_folder}_histogram_frame_{frame_id}.csv')
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([f'Bin {i+1}' for i in range(n)])  # Write header
        for vector in histogram_vectors:
            writer.writerow(vector)

    print(f"Histogram vectors saved to {csv_file}")

Histogram vectors saved to /home/vdixit5/Desktop/MWD project files/Target_video/Wave_output/Chris_Benoit_waving_goodbye_wave_u_nm_np1_fr_med_1_output_histogram_frame_1.csv
Histogram vectors saved to /home/vdixit5/Desktop/MWD project files/Target_video/Wave_output/Chris_Benoit_waving_goodbye_wave_u_nm_np1_fr_med_1_output_histogram_frame_2.csv
Histogram vectors saved to /home/vdixit5/Desktop/MWD project files/Target_video/Wave_output/Chris_Benoit_waving_goodbye_wave_u_nm_np1_fr_med_1_output_histogram_frame_3.csv
Histogram vectors saved to /home/vdixit5/Desktop/MWD project files/Target_video/Wave_output/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_7_output_histogram_frame_1.csv
Histogram vectors saved to /home/vdixit5/Desktop/MWD project files/Target_video/Wave_output/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_7_output_histogram_frame_2.csv
Histogram vectors saved to /ho